# Preprocessing Pipeline

In [3]:
# necessary imports
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

/Users/mauro/.pyenv/versions/3.10.6/envs/sandbox/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Data Extraction

In [4]:
# Methods and  lists for data  extraction

def df_to_timeseries(df, timestamp_col="timestamp"):
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], unit="ms") # convert timestamp to datetime
    df.set_index(timestamp_col,inplace=True) # set it as index for TimeSeries purposes
    df = df.tz_localize("UTC") # need to localize a timezone
    df = df.tz_convert("Asia/Seoul") # convert it to Korean timezone
    return df

def get_home_coor(amb_df,loc_df):
    time_diff = amb_df.index.to_series().diff()
    # Find indices where the time difference exceeds 2 hours
    jump_indices = time_diff[time_diff > pd.Timedelta(hours=2)].index
    closest_indices = {}
    for jump_index in jump_indices:
        indexer = loc_df.index.get_indexer([jump_index])
        closest_index = loc_df.index[indexer[0]]
        closest_indices[jump_index] = closest_index

    # Extract values from other_df at the closest indices and get their average
    return loc_df.loc[closest_indices.values()].mean()

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c
    return km * 1000



In [16]:
# Create the mapping dictionary
app_name_to_new_category = {
    'Netflix': 'VIDEO_PLAYER',
    'Galaxy Store': 'SHOPPING',
    'AfreecaTV': 'VIDEO_PLAYER',
    '카카오페이지': 'BOOKS_AND_REFERENCE',
    'Google Play 게임': 'GAME_RELATED',
    '롯데시네마': 'SHOPPING',
    '네이버TV': 'VIDEO_PLAYER',
    'Twitch': 'VIDEO_PLAYER',
    '어드벤처': 'GAME_RELATED',
    'CGV': 'SHOPPING',
    '연애의과학': 'BOOKS_AND_REFERENCE',
    '팟빵': 'MUSIC_AND_AUDIO',
    'U+모바일tv': 'VIDEO_PLAYER',
    'TVING': 'VIDEO_PLAYER',
    'JAM Live': 'VIDEO_PLAYER',
    'CashLeaflet': 'SHOPPING',
    '티켓링크': 'SHOPPING',
    'Galaxy Apps': 'SHOPPING',
    'LoL 상점': 'GAME_RELATED',
    'OP.GG': 'GAME_RELATED',
    'GGtics': 'GAME_RELATED',
    'CGV포토티켓': 'SHOPPING',
    'Prime Video': 'VIDEO_PLAYER',
    '왓챠': 'VIDEO_PLAYER',
    '인터파크 티켓': 'SHOPPING',
    '왓챠플레이': 'VIDEO_PLAYER',
    'Doctor Who': 'VIDEO_PLAYER',
    '4shared': 'FILE_SHARING',
    '팝콘티비': 'VIDEO_PLAYER',
    '얼굴인식 체험판': 'LIBRARIES_AND_DEMO',
    '올레 tv 모바일': 'VIDEO_PLAYER',
    '아이즈원 프라이빗 메일': 'FAN_COMMUNITY',
    'V LIVE': 'SOCIAL',
    'FOW.KR': 'GAME_RELATED',
    'Steam': 'GAME_RELATED',
    '메가박스': 'MOVIE_THEATER',
    'Nintendo Switch Online': 'GAME_RELATED',
    '피키캐스트': 'SOCIAL',
    'HTV 3.4.6': 'VIDEO_PLAYER',
    'Q.Feat': 'VIDEO_PLAYER',
    'WoW BfA Talent Calculator': 'GAME_RELATED',
    '꽁음따 시즌4': 'GAME_RELATED',
    
}

# Define a function to map app names to new categories
def map_entertainment_to_new_category(row):
    if row["category"] == "ENTERTAINMENT":
        return app_name_to_new_category.get(row["name"], "ENTERTAINMENT")
    elif row["category"] == "MISC":
        return app_name_to_new_category.get(row["name"], "MISC")
    else:
        return row["category"]


In [21]:
data_dir = os.path.join(os.getcwd() , "data")
data_files = [f"{name}.csv" for name in ["AppUsageEvent", "Location", "AmbientLight", "UltraViolet"]]
column_names = ["app_name", "app_category", "brightness", "uv_intensity", "longitude", "latitude","altitude"]
entertainment_categories = ['SOCIAL','SHOPPING','BOOKS_AND_REFERENCE',
                            'COMICS','MUSIC_AND_AUDIO', 'GAME','ENTERTAINMENT','VIDEO_PLAYERS', 
                            'SPORTS','NEWS_AND_MAGAZINES']#'COMMUNICATION'?

esm_path = os.path.join(data_dir,"SubjData","EsmResponse.csv")
esm_df = df_to_timeseries(pd.read_csv(esm_path),timestamp_col="responseTime")[["valence","arousal","attention","stress","pcode"]]
P = {} # Dictionary that'll contain all participant data

for p_code in os.listdir(data_dir):
    if p_code.startswith("P"): # Only get Participant Directories
        pn = int(p_code[1:])
        
        # get tables (and columns) of interest
        app_df = df_to_timeseries(pd.read_csv(os.path.join(data_dir,p_code, "AppUsageEvent.csv")))[["name","category"]]
        # Apply the function to update the app_category column
        app_df["category"] = app_df.apply(map_entertainment_to_new_category, axis=1)
        app_df = app_df[app_df["category"].isin(entertainment_categories)] # filter for entertainment related apps only
        amb_df = df_to_timeseries(pd.read_csv(os.path.join(data_dir,p_code, "AmbientLight.csv")))
        uv_df = df_to_timeseries(pd.read_csv(os.path.join(data_dir,p_code, "UltraViolet.csv")))[["intensity"]]
        loc_df = df_to_timeseries(pd.read_csv(os.path.join(data_dir,p_code, "Location.csv")))
        loc_df = loc_df[loc_df["accuracy"] < 20][["longitude","latitude","altitude"]] # remove innacurate readings
        
        # merge using merge_asof (we match on nearest timestamp rather than equal timestamps)
        joined_df = pd.merge_asof(app_df, amb_df, left_index=True,right_index=True)
        joined_df = pd.merge_asof(joined_df, uv_df, left_index=True,right_index=True)
        joined_df = pd.merge_asof(joined_df, loc_df, left_index=True,right_index=True)
        joined_df.columns = column_names
        
        # remove duplicates and nan values
        df = joined_df.drop_duplicates().dropna()
        
        # calculate distance from determined home
        home_longitude,home_latitude,_ = get_home_coor(amb_df,loc_df)
        distances = haversine(joined_df['longitude'], df['latitude'], home_longitude, home_latitude)
        joined_df["at_home"] = distances <= 25
        
        P[pn] = pd.merge_asof(joined_df, esm_df[esm_df["pcode"] == p_code], left_index=True,right_index=True).drop("pcode",axis=1)

In [12]:
# Example of a table
P[10].sample(10)

,app_name,app_category,brightness,uv_intensity,longitude,latitude,altitude,at_home,valence,arousal,attention,stress
timestamp,,,,,,,,,,,,
2019-05-10 22:01:28.256000+09:00,네이버 웹툰,COMICS,0.0,NONE,33.820208,37.495659,0.000000,False,1.0,1.0,1.0,-1.0
2019-05-10 21:46:54.606000+09:00,네이버 웹툰,COMICS,0.0,NONE,33.820240,37.495734,0.000000,False,1.0,1.0,1.0,-1.0
2019-05-10 17:17:05.375000+09:00,네이버 카페,SOCIAL,7.0,NONE,33.820235,37.495742,0.000000,False,1.0,1.0,1.0,-1.0
2019-05-10 23:21:41.426000+09:00,네이버 웹툰,COMICS,53.0,NONE,33.820235,37.495720,0.000000,False,1.0,1.0,1.0,-1.0
2019-05-10 13:33:46.993000+09:00,네이버 카페,SOCIAL,50.0,NONE,34.321431,36.685498,73.888610,False,-1.0,0.0,0.0,1.0
2019-05-11 14:16:26.782000+09:00,네이버 카페,SOCIAL,8.0,NONE,33.820224,37.495755,0.000000,False,1.0,2.0,1.0,0.0
2019-05-12 11:08:28.112000+09:00,네이버 카페,SOCIAL,73.0,NONE,34.314234,36.370641,79.599998,False,1.0,-1.0,-1.0,-2.0
2019-05-14 08:27:33.563000+09:00,네이버 카페,SOCIAL,21.0,NONE,34.314223,36.370587,85.599998,False,1.0,1.0,1.0,1.0
2019-05-10 13:29:01.041000+09:00,네이버 카페,SOCIAL,1.0,NONE,34.345266,36.618930,80.003213,False,-1.0,0.0,0.0,1.0


## Saving preprocessed data

In [9]:
!mkdir clean_data

In [19]:
for number, participant_df in P.items():
    participant_df.to_csv(f"clean_data/P{number}.csv")

## Descriptive statistics

In [22]:
combined_df = pd.concat(P.values(), keys=P.keys()) # combine all partcipant data

combined_df[combined_df["at_home"]==True].describe()

,brightness,longitude,latitude,altitude,valence,arousal,attention,stress
count,7236.000000,7236.000000,7236.000000,7236.000000,7035.000000,7035.000000,7035.000000,7035.000000
mean,128.286761,34.344248,36.348600,84.664014,0.608813,-0.370576,0.320682,-0.264677
std,546.506235,0.210790,0.166427,26.142302,1.318352,1.589475,1.497441,1.606037
min,0.000000,34.289438,35.096366,-83.906865,-3.000000,-3.000000,-3.000000,-3.000000
25%,15.000000,34.314563,36.368074,76.174997,0.000000,-2.000000,-1.000000,-2.000000
50%,48.000000,34.315287,36.370611,89.500000,1.000000,0.000000,0.000000,0.000000
75%,113.000000,34.317485,36.373941,95.699997,2.000000,1.000000,1.000000,1.000000
max,16756.000000,36.018408,36.408144,232.811909,3.000000,3.000000,3.000000,3.000000


In [23]:
combined_df["brightness"].describe()

count    189423.000000
mean        158.785095
std        1176.328104
min           0.000000
25%           3.000000
50%          29.000000
75%          85.000000
max       65535.000000
Name: brightness, dtype: float64

In [6]:
combined_df[combined_df["at_home"]==False].describe()

,brightness,longitude,latitude,altitude,valence,arousal,attention,stress
count,188695.000000,190095.000000,190095.000000,190095.000000,188310.000000,188310.000000,188310.000000,188310.000000
mean,157.021527,34.295346,36.417877,80.673210,0.610844,-0.184085,0.277526,-0.462546
std,1174.468743,2.343973,0.355026,28.361543,1.334560,1.554549,1.566323,1.561716
min,0.000000,-243.254757,35.096366,-132.824455,-3.000000,-3.000000,-3.000000,-3.000000
25%,3.000000,34.314800,36.368086,75.000000,0.000000,-2.000000,-1.000000,-2.000000
50%,29.000000,34.316165,36.370717,84.500000,1.000000,0.000000,0.000000,-1.000000
75%,82.000000,34.320572,36.373927,95.300003,2.000000,1.000000,1.000000,1.000000
max,65535.000000,36.250271,37.709008,1151.584606,3.000000,3.000000,3.000000,3.000000


In [7]:
combined_df[combined_df["app_category"]=="SOCIAL"].describe()

,brightness,longitude,latitude,altitude,valence,arousal,attention,stress
count,108419.000000,109219.000000,109219.000000,109219.000000,107897.000000,107897.000000,107897.000000,107897.000000
mean,154.765189,34.314069,36.415923,81.468629,0.600758,-0.225215,0.206929,-0.589043
std,1217.406928,0.300301,0.370142,28.963387,1.280058,1.516711,1.551036,1.527141
min,0.000000,33.333897,35.096394,-83.906865,-3.000000,-3.000000,-3.000000,-3.000000
25%,3.000000,34.314659,36.368045,75.000000,0.000000,-2.000000,-1.000000,-2.000000
50%,25.000000,34.315782,36.370767,84.699997,1.000000,0.000000,0.000000,-1.000000
75%,78.000000,34.319892,36.373878,95.500000,2.000000,1.000000,1.000000,1.000000
max,65535.000000,36.250271,37.709008,1151.584606,3.000000,3.000000,3.000000,3.000000


In [8]:
combined_df[combined_df["app_category"]=="GAME"].mean()

TypeError: Could not convert ['히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레Duel LinksDuel Links히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레히어로칸타레히어로칸타레Duel LinksDuel Links히어로칸타레히어로칸타레히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel Links히어로칸타레히어로칸타레Duel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel LinksDuel Link

In [ ]:
combined_df[combined_df["app_category"]=="COMICS"].describe()

## Data Exploration

In [ ]:
pn = 55
# Calculate category counts for home and other locations
home_category_counts = P[pn][P[pn]["at_home"]==True]['app_category'].value_counts()
other_category_counts = P[pn][P[pn]["at_home"]==False]['app_category'].value_counts()

# Determine unique categories and their counts across both home and other locations
all_categories = set(home_category_counts.index) | set(other_category_counts.index)
max_count = max(home_category_counts.max(), other_category_counts.max())

# Define a custom color palette
custom_palette = {
    'SOCIAL': '#6EB5FF',             # Light blue
    'SHOPPING': '#FFB347',           # Light orange
    'BOOKS_AND_REFERENCE': '#9ACD32',# Yellow-green
    'COMICS': '#FF6347',             # Salmon
    'MUSIC_AND_AUDIO': '#BA55D3',    # Lavender
    'GAME': '#CD853F',               # Peru
    'ENTERTAINMENT': '#FF69B4',      # Hot pink
    'VIDEO_PLAYERS': '#20B2AA',      # Light sea green
    'SPORTS': '#808000',             # Olive
    'LIBRARIES_AND_DEMO': '#BEBEBE', # Light gray
    'NEWS_AND_MAGAZINES': '#FFD700'  # Gold
}


# Create subplots with 1 row and 2 columns
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot for home category counts
sns.barplot(x=home_category_counts.index, y=home_category_counts.values, palette=custom_palette, ax=axes[0])
axes[0].set_title('Home Location')
axes[0].set_xlabel('App Category')
axes[0].set_ylabel('Count')

# Plot for other location category counts
sns.barplot(x=other_category_counts.index, y=other_category_counts.values, palette=custom_palette, ax=axes[1])
axes[1].set_title('Other Location')
axes[1].set_xlabel('App Category')
axes[1].set_ylabel('Count')

plt.legend(custom_palette.keys(), custom_palette.values())


# Adjust layout
plt.tight_layout()

# Show plots
plt.show()